# Data Exploration

Before writing any parser, let's understand the actual data.

In [1]:
import json
import sys
from pathlib import Path

import pandas as pd
from pypdf import PdfReader

# Add project root to path
sys.path.insert(0, str(Path.cwd().parent))
from src.config import settings

print(f"Dataset root: {settings.dataset_root}")
print(f"Menu dir: {settings.menu_dir}")

Dataset root: /Users/sara.callaioli/projects/datapizza_assignement/Dataset
Menu dir: /Users/sara.callaioli/projects/datapizza_assignement/Dataset/knowledge_base/menu


## 1. Dataset overview

In [2]:
# Questions
questions_df = pd.read_csv(settings.questions_csv_path)
print(f"Questions: {len(questions_df)}")
print(f"Columns: {list(questions_df.columns)}")
print(f"\nDifficulty distribution:")
print(questions_df["difficoltà"].value_counts())
questions_df.head()

Questions: 100
Columns: ['domanda', 'difficoltà']

Difficulty distribution:
difficoltà
Easy          48
Medium        28
Hard          18
Impossible     6
Name: count, dtype: int64


,domanda,difficoltà
0,Quali sono i piatti che includono le Chocobo W...,Easy
1,Quali piatti dovrei scegliere per un banchetto...,Easy
2,Quali sono i piatti della galassia che conteng...,Easy
3,Quali piatti contengono i Ravioli al Vaporeon?,Easy
4,Quali sono i piatti che includono i Sashimi di...,Easy


In [6]:
# Dish mapping
with open(settings.dish_mapping_path) as f:
    dish_mapping = json.load(f)

print(f"Total dishes: {len(dish_mapping)}")
print(f"ID range: {min(dish_mapping.values())} - {max(dish_mapping.values())}")
print(f"\nFirst 10 dishes:")
for name, id_ in list(dish_mapping.items())[:10]:
    print(f"  {id_:3d}: {name}")

# Data quality checks
from collections import Counter

print("\n" + "="*80)
print("DATA QUALITY CHECKS")
print("="*80)

# Check for duplicate IDs
id_counts = Counter(dish_mapping.values())
duplicate_ids = {id_: count for id_, count in id_counts.items() if count > 1}

# Check for duplicate names (case-insensitive)
name_lower = [name.lower() for name in dish_mapping.keys()]
name_counts = Counter(name_lower)
duplicate_names = {name: count for name, count in name_counts.items() if count > 1}

# Check for missing IDs in range 0-286
all_ids = set(dish_mapping.values())
expected_ids = set(range(287))
missing_ids = expected_ids - all_ids

print(f"\nUnique IDs: {len(set(dish_mapping.values()))} / {len(dish_mapping)}")
print(f"Unique names (case-sensitive): {len(set(dish_mapping.keys()))} / {len(dish_mapping)}")
print(f"Unique names (case-insensitive): {len(set(name_lower))} / {len(dish_mapping)}")

print(f"\n✓ Duplicate IDs: {len(duplicate_ids)}")
if duplicate_ids:
    for id_, count in sorted(duplicate_ids.items()):
        names = [name for name, i in dish_mapping.items() if i == id_]
        print(f"  ID {id_} appears {count} times: {names}")

print(f"✓ Duplicate names (case-insensitive): {len(duplicate_names)}")
if duplicate_names:
    for name, count in sorted(duplicate_names.items()):
        print(f"  '{name}' appears {count} times")

print(f"✓ Missing IDs in range 0-286: {len(missing_ids)}")
if missing_ids:
    print(f"  Missing: {sorted(missing_ids)}")

Total dishes: 287
ID range: 0 - 286

First 10 dishes:
    0: Alternate Realities Risotto
    1: Antipasto Celestiale
    2: Antipasto Stellare dell'Eterna Armonia
    3: Armonia Cosmica alla Tavola d'Oro
    4: Armonia Cosmica della Fenice
    5: Astro-Risotto alle Onde Temporali
    6: Aurora del Cosmo
    7: Bistecca Cacofonica dell'Infinito
    8: Concordanza Cosmica
    9: Cosmic Harmony

DATA QUALITY CHECKS

Unique IDs: 287 / 287
Unique names (case-sensitive): 287 / 287
Unique names (case-insensitive): 287 / 287

✓ Duplicate IDs: 0
✓ Duplicate names (case-insensitive): 0
✓ Missing IDs in range 0-286: 0


In [7]:
# Ground truth
gt_df = pd.read_csv(settings.ground_truth_csv_path)
print(f"Ground truth rows: {len(gt_df)}")
print(f"Columns: {list(gt_df.columns)}")
print(f"\nUsage split:")
print(gt_df["Usage"].value_counts())

# Answer size distribution
gt_df["n_dishes"] = gt_df["result"].astype(str).apply(lambda x: len(x.split(",")))
print(f"\nDishes per answer: min={gt_df['n_dishes'].min()}, max={gt_df['n_dishes'].max()}, mean={gt_df['n_dishes'].mean():.1f}")
gt_df.head(10)

Ground truth rows: 100
Columns: ['row_id', 'result', 'Usage']

Usage split:
Usage
Private    50
Public     50
Name: count, dtype: int64

Dishes per answer: min=1, max=10, mean=3.7


,row_id,result,Usage,n_dishes
0,1,78,Private,1
1,2,225,Public,1
2,3,156,Private,1
3,4,215,Public,1
4,5,94,Private,1
5,6,179,Public,1
6,7,"171,267,189",Private,3
7,8,"13,130,15,6,51,209",Public,6
8,9,"207,76",Private,2
9,10,"184,115,266",Public,3


## 2. Menu PDF structure

In [8]:
# List all menu PDFs
pdf_files = sorted(settings.menu_dir.glob("*.pdf"))
print(f"Menu PDFs: {len(pdf_files)}")
for p in pdf_files:
    size_kb = p.stat().st_size / 1024
    print(f"  {size_kb:6.1f} KB  {p.name}")

Menu PDFs: 30
    53.1 KB  Anima Cosmica.pdf
    30.7 KB  Armonia Universale.pdf
    29.8 KB  Cosmica Essenza.pdf
  1373.8 KB  Datapizza.pdf
    53.6 KB  Eco di Pandora.pdf
    57.8 KB  Eredita Galattica.pdf
    42.0 KB  Essenza dell Infinito.pdf
    44.1 KB  Il Firmamento.pdf
    49.1 KB  L Architetto dell Universo.pdf
    49.5 KB  L Eco dei Sapori.pdf
    54.1 KB  L Equilibrio Quantico.pdf
    48.8 KB  L Essenza Cosmica.pdf
    47.6 KB  L Essenza del Multiverso su Pandora.pdf
   255.4 KB  L Essenza delle Dune.pdf
    37.5 KB  L Essenza di Asgard.pdf
   605.8 KB  L Etere del Gusto.pdf
    57.0 KB  L Oasi delle Dune Stellari.pdf
    47.5 KB  L Universo in Cucina.pdf
    50.4 KB  L infinito in un Boccone.pdf
   376.4 KB  Le Dimensioni del Gusto.pdf
   195.5 KB  Le Stelle Danzanti.pdf
    52.7 KB  Le Stelle che Ballano.pdf
    40.8 KB  Ristorante Quantico.pdf
    53.6 KB  Ristorante delle Dune Stellari.pdf
    53.3 KB  Sala del Valhalla.pdf
    30.4 KB  Sapore del Dune.pdf
    45.1 KB  S

In [9]:
# Parse a structured menu (bullet-list format)
reader = PdfReader(settings.menu_dir / "Sapore del Dune.pdf")
text = "\n".join(page.extract_text() for page in reader.pages)
print(f"Pages: {len(reader.pages)}")
print(f"Total chars: {len(text)}")
print("="*80)
print(text[:3000])

Pages: 5
Total chars: 5640
Ristorante "Sapore del Dune"
Chef Alessandra Quanti
Nel cuore arido di Tatooine, dove i mondi si mescolano e le stelle guidano i viaggiatori intergalattici, Chef
Alessandra Quanti porta una rivoluzione culinaria che sfida le distanze siderali. Non è raro vedere i
commensali rimanere incantati osservando i suoi piatti che sembrano danzare tra le dune e le stelle, frutto
della sua straordinaria padronanza degli stati quantici, che le permette di esplorare e materializzare le infinite
possibilità nascoste in ogni ingrediente rarefatto del deserto.
La sua storia ebbe inizio nei laboratori di spezie di Mos Eisley, dove la passione per la gastronomia
molecolare si fuse con la sua profonda comprensione dell'universo subatomico. Fu proprio durante un
esperimento particolarmente intenso con i Cristalli Kyber che scoprì la sua innata capacità di percepire le
probabilità culinarie, un dono che trasformò ogni sua creazione in un'esperienza di perfezione matematica
anche 

In [10]:
# Parse Datapizza.pdf (narrative prose format - different structure!)
reader2 = PdfReader(settings.menu_dir / "Datapizza.pdf")
text2 = "\n".join(page.extract_text() for page in reader2.pages)
print(f"Pages: {len(reader2.pages)}")
print(f"Total chars: {len(text2)}")
print("="*80)
print(text2[:3000])

Pages: 6
Total chars: 18423
Ristorante "L'Infinito Sapore"
Viaggio nel Tempo e nel Gusto su Pandora
Chef Alessandro-Pierpaolo-Jack Quantum
Sotto i cieli incantevoli di Pandora, dove le montagne fluttuano tra le nuvole bioluminescenti, si apre un
portale verso esperienze culinarie senza confini. Qui, all'Infinito Sapore, lo Chef Alessandro-Pierpaolo-
Jack, una curiosa chimera che ha tre stati quantici in superposizione che ha raggiunto superintellinza, ha
deciso di aprire un ristorante startup chiamato Datapizza.
Il suo straordinario viaggio iniziò con la fisica quantistica, una passione che si fuse con l'arte della cucina.
Questa combinazione unica gli conferisce una maestria della Quantistica (EDUCATION di livello 11), che
trasforma ogni sua creazione in un'opera multidimensionale, esistente simultaneamente in undici stati,
pronte a essere scelte dall'osservatore al momento perfetto.
La sua abilità nel manipolare il tessuto temporale (Education Level Temporale II) si rivela nella cura

In [11]:
# Parse a third menu for comparison
reader3 = PdfReader(settings.menu_dir / "L Essenza di Asgard.pdf")
text3 = "\n".join(page.extract_text() for page in reader3.pages)
print(f"Pages: {len(reader3.pages)}")
print(f"Total chars: {len(text3)}")
print("="*80)
print(text3[:3000])

Pages: 6
Total chars: 10443
Ristorante: L'Essenza di Asgard
Chef Palissandro "Sandro" Luminetti
Nel regno scintillante di Asgard, dove il Bifrost collega mondi lontani e il cielo racconta storie di eroi e dèi, lo
Chef Palissandro "Sandro" Luminetti emerge come una forza divina della creazione culinaria. Ogni giorno, il
suo talento esplode come un'enorme supernova, trasformando la cucina in una danza di energia cosmica.
Non è raro vederlo in una contemplazione profonda davanti ai suoi mistici fornelli multicelesti, mentre il suo
spirito esplora infinite realtà parallele vantando di un LTK ben oltre superiore al VI, plasmando le sue
creazioni in delicate sinfonie di essenze primarie.
La sua arte culinaria si fonda su una connessione psichica certificata di grado II con gli ingredienti più rari e
mitici. Le sue mani si muovono con una grazia eterea, orchestrando dialoghi silenziosi tra spezie elfiche,
frutti delle sacre foreste e carni di creature leggendarie. Ogni ingrediente svela la su

## 3. Supporting documents

In [12]:
# Distances matrix
distances_df = pd.read_csv(settings.distances_csv_path, index_col=0)
print(f"Planets: {list(distances_df.columns)}")
distances_df

Planets: ['Tatooine', 'Asgard', 'Namecc', 'Arrakis', 'Krypton', 'Pandora', 'Cybertron', 'Ego', 'Montressosr', 'Klyntar']


,Tatooine,Asgard,Namecc,Arrakis,Krypton,Pandora,Cybertron,Ego,Montressosr,Klyntar
/,,,,,,,,,,
Tatooine,0,695,641,109,661,1130,344,835,731,530
Asgard,695,0,550,781,188,473,493,156,240,479
Namecc,641,550,0,651,367,987,728,688,767,845
Arrakis,109,781,651,0,727,1227,454,926,834,640
Krypton,661,188,367,727,0,626,557,321,422,599
Pandora,1130,473,987,1227,626,0,847,317,413,731
Cybertron,344,493,728,454,557,847,0,594,434,186
Ego,835,156,688,926,321,317,594,0,215,532
Montressosr,731,240,767,834,422,413,434,215,0,331


In [14]:
# Manuale di Cucina (technique categories)
manual = PdfReader(settings.manuale_path)
manual_text = "\n".join(page.extract_text() for page in manual.pages)
print(f"Pages: {len(manual.pages)}")
print(f"Total chars: {len(manual_text)}")
print("="*80)
print(manual_text[:1000])

Pages: 20
Total chars: 52051
Introduzione
Io sono il grande Sirius Cosmo, lo chef stellare per eccellenza. Se non mi conoscete ancora, preparatevi:
il mio nome è sinonimo di cucina galattica. In questo manuale vi insegnerò tutto quello che serve per
diventare veri cuochi, da Alpha Centauri fino alla Nebulosa del Granchio (dove, a proposito, non hanno
neanche un crostaceo decente). Impareremo insieme le licenze e le abilità fondamentali per cucinare
nello spazio senza mandare in tilt lʼintero sistema di supporto vitale della nave — perché diciamocelo, un
buon brasato non vale una depressurizzazione dʼemergenza.
Vi guiderò passo passo tra tecniche di cucina gravitazionali, segreti per friggere senza far esplodere la
cabina e trucchi per ottenere il perfetto soufflé orbitale.
Spoiler: Sì, si può montare una maionese in assenza di gravità — basta avere il giusto polso e un poʼ di
pazienza, cose che non si insegnano nei manuali tecnici ma che io, Sirius Cosmo, sono qui per
trasmettervi.
Pro

## 4. Quick parsing test

Can we extract dish names from the structured menu format?

In [15]:
import re

# For structured menus like "Sapore del Dune", dishes appear as bold headers
# followed by Ingredienti/Tecniche sections.
# In raw text, dish names appear on their own line before "Ingredienti"

# Split text by "Ingredienti" to find dish blocks
blocks = re.split(r'(?=Ingredienti)', text)
print(f"Found {len(blocks)} blocks with 'Ingredienti'")

for i, block in enumerate(blocks[:3]):
    print(f"\n--- Block {i} (first 300 chars) ---")
    print(block[:300])

Found 11 blocks with 'Ingredienti'

--- Block 0 (first 300 chars) ---
Ristorante "Sapore del Dune"
Chef Alessandra Quanti
Nel cuore arido di Tatooine, dove i mondi si mescolano e le stelle guidano i viaggiatori intergalattici, Chef
Alessandra Quanti porta una rivoluzione culinaria che sfida le distanze siderali. Non è raro vedere i
commensali rimanere incantati osserv

--- Block 1 (first 300 chars) ---
Ingredienti
Shard di Prisma Stellare
Lattuga Namecciana
Radici di Singolarità
Fibra di Sintetex
Carne di Balena spaziale
Teste di Idra
Nettare di Sirena
Sale Temporale
Tecniche
Marinatura Temporale Sincronizzata
Cottura Sottovuoto Frugale Energeticamente Negativa
Cottura a Vapore Termocinetica Multi

--- Block 2 (first 300 chars) ---
Ingredienti
Shard di Prisma Stellare
Foglie di Nebulosa
Lattuga Namecciana
Teste di Idra
Carne di Mucca
Farina di Nettuno
Riso di Cassandra
Fusilli del Vento
Nduja Fritta Tanto
Tecniche
Cottura Sottovuoto Frugale Energeticamente Negativa
Cottura a Vapore Ter

In [16]:
# Check: which dish names from dish_mapping appear in this menu's text?
found_in_mapping = []
for dish_name in dish_mapping:
    if dish_name in text:
        found_in_mapping.append(dish_name)

print(f"Dish names from mapping found in 'Sapore del Dune': {len(found_in_mapping)}")
for name in found_in_mapping:
    print(f"  - {name} (ID: {dish_mapping[name]})")

Dish names from mapping found in 'Sapore del Dune': 12
  - Evanescenza Quantica (ID: 47)
  - Galassia di Sapore (ID: 68)
  - Galassia di Sapore Quantico (ID: 71)
  - Galassia di Sapori (ID: 73)
  - Galassia di Sapori Sublimi (ID: 76)
  - Interstellare Risveglio di Kraken (ID: 106)
  - Ode Cosmica di Terra e Stelle (ID: 141)
  - Pioggia di Dimensioni Galattiche (ID: 152)
  - Sinfonia Galattica (ID: 230)
  - Sinfonia Quantistica dell'Universo (ID: 239)
  - Sinfonia Quantistica delle Stelle (ID: 240)
  - Sinfonia Temporale delle Profondità Infrasoniche (ID: 245)


In [17]:
# Same check for Datapizza.pdf
found_in_datapizza = []
for dish_name in dish_mapping:
    if dish_name in text2:
        found_in_datapizza.append(dish_name)

print(f"Dish names from mapping found in 'Datapizza': {len(found_in_datapizza)}")
for name in found_in_datapizza:
    print(f"  - {name} (ID: {dish_mapping[name]})")

Dish names from mapping found in 'Datapizza': 9
  - Pizza Baby Daniele (ID: 153)
  - Pizza Baby Lorenzo (ID: 154)
  - Pizza Baby Simone e Alessandro (ID: 155)
  - Pizza Cri (ID: 157)
  - Pizza Emma (ID: 158)
  - Pizza Fra (ID: 159)
  - Pizza Gio (ID: 160)
  - Pizza Luca (ID: 161)
  - Pizza Raul (ID: 162)


In [18]:
# Check ALL menus - how many of the 287 dishes can we find by exact name match?
all_found = set()
dishes_per_menu = {}

for pdf_path in pdf_files:
    reader = PdfReader(pdf_path)
    full_text = "\n".join(page.extract_text() for page in reader.pages)
    
    menu_dishes = []
    for dish_name in dish_mapping:
        if dish_name in full_text:
            all_found.add(dish_name)
            menu_dishes.append(dish_name)
    
    dishes_per_menu[pdf_path.stem] = len(menu_dishes)
    
print(f"Total dishes found by exact name match: {len(all_found)} / {len(dish_mapping)}")
print(f"Missing: {len(dish_mapping) - len(all_found)}")
print(f"\nDishes per menu:")
for menu, count in sorted(dishes_per_menu.items()):
    print(f"  {count:2d} dishes  {menu}")

Total dishes found by exact name match: 280 / 287
Missing: 7

Dishes per menu:
  12 dishes  Anima Cosmica
  11 dishes  Armonia Universale
  13 dishes  Cosmica Essenza
   9 dishes  Datapizza
  12 dishes  Eco di Pandora
  10 dishes  Eredita Galattica
  10 dishes  Essenza dell Infinito
   7 dishes  Il Firmamento
  11 dishes  L Architetto dell Universo
  12 dishes  L Eco dei Sapori
   9 dishes  L Equilibrio Quantico
  11 dishes  L Essenza Cosmica
  11 dishes  L Essenza del Multiverso su Pandora
  10 dishes  L Essenza delle Dune
  12 dishes  L Essenza di Asgard
  12 dishes  L Etere del Gusto
   9 dishes  L Oasi delle Dune Stellari
  10 dishes  L Universo in Cucina
  11 dishes  L infinito in un Boccone
  13 dishes  Le Dimensioni del Gusto
   9 dishes  Le Stelle Danzanti
  11 dishes  Le Stelle che Ballano
  12 dishes  Ristorante Quantico
  12 dishes  Ristorante delle Dune Stellari
   9 dishes  Sala del Valhalla
  12 dishes  Sapore del Dune
   9 dishes  Stelle Astrofisiche
   9 dishes  Stelle 

In [20]:
# Which dishes are missing with raw exact match?
missing = set(dish_mapping.keys()) - all_found
print(f"Missing dishes with raw exact match: {len(missing)}")
for name in sorted(missing):
    print(f"  - {name} (ID: {dish_mapping[name]})")

Missing dishes with raw exact match: 7
  - Mandragola e Radici (ID: 120)
  - Piastrella Celestiale di Gnocchi del Crepuscolo con Nebulosa di Riso di Cassandra, Lacrime di Unicorno e Velo di Materia Oscura (ID: 148)
  - Pizza Cosmica all'Essenza di Drago con Nebbia Arcobaleno e Funghi Orbitali (ID: 156)
  - Portale Cosmico: Sinfonia di Gnocchi del Crepuscolo con Essenza di Tachioni e Sfumature di Fenice (ID: 169)
  - Risotto Interdimensionale alla Carne di Drago e Balena Spaziale con Biscotti della Galassia Croccanti (ID: 183)
  - Sinfonia Quantica dell'Oceano Interstellare (ID: 238)
  - Sinfonia Temporale di Fenice e Xenodonte su Pane degli Abissi con Colata di Plasma Vitale e Polvere di Crononite (ID: 246)


## 5. Text normalization — recovering missing dishes

Why this matters: the submission requires **dish IDs** from `dish_mapping.json`. If we can locate dish names exactly in the PDF text, we can use them as **anchors** to split text into per-dish blocks and extract ingredients/techniques. No fuzzy matching or LLM-based detection needed.

The raw exact match above misses some dishes. Two causes:
1. **Line wrapping**: long dish names break across PDF lines (`\n` inserted mid-name)
2. **Curly quotes**: pypdf extracts `'` (U+2019) while `dish_mapping.json` uses `'` (U+0027)

In [21]:
# Step 1: Whitespace normalization (collapse \n into spaces)
def normalize_text(text):
    """Replace newlines with spaces and collapse multiple spaces."""
    return " ".join(text.split())

all_found_ws = set()
for pdf_path in pdf_files:
    reader = PdfReader(pdf_path)
    full_text = normalize_text("\n".join(page.extract_text() for page in reader.pages))
    for dish_name in dish_mapping:
        if dish_name in full_text:
            all_found_ws.add(dish_name)

recovered_ws = all_found_ws - all_found
still_missing_ws = set(dish_mapping.keys()) - all_found_ws

print(f"After whitespace normalization: {len(all_found_ws)} / {len(dish_mapping)}")
print(f"Recovered {len(recovered_ws)} dishes:")
for name in sorted(recovered_ws):
    print(f"  + {name} (ID: {dish_mapping[name]})")
print(f"\nStill missing: {len(still_missing_ws)}")
for name in sorted(still_missing_ws):
    print(f"  - {name} (ID: {dish_mapping[name]})")

After whitespace normalization: 285 / 287
Recovered 5 dishes:
  + Piastrella Celestiale di Gnocchi del Crepuscolo con Nebulosa di Riso di Cassandra, Lacrime di Unicorno e Velo di Materia Oscura (ID: 148)
  + Pizza Cosmica all'Essenza di Drago con Nebbia Arcobaleno e Funghi Orbitali (ID: 156)
  + Portale Cosmico: Sinfonia di Gnocchi del Crepuscolo con Essenza di Tachioni e Sfumature di Fenice (ID: 169)
  + Risotto Interdimensionale alla Carne di Drago e Balena Spaziale con Biscotti della Galassia Croccanti (ID: 183)
  + Sinfonia Temporale di Fenice e Xenodonte su Pane degli Abissi con Colata di Plasma Vitale e Polvere di Crononite (ID: 246)

Still missing: 2
  - Mandragola e Radici (ID: 120)
  - Sinfonia Quantica dell'Oceano Interstellare (ID: 238)


In [ ]:
# Step 2: Curly quote normalization
# Some dish names in dish_mapping.json use curly quotes (\u2019), some use straight (').
# PDFs also mix both. Normalize BOTH sides to straight apostrophes for matching.
def normalize_quotes(text):
    """Replace curly/smart quotes with straight apostrophes."""
    return text.replace("\u2019", "'").replace("\u2018", "'")

all_found_full = set()
for pdf_path in pdf_files:
    reader = PdfReader(pdf_path)
    full_text = normalize_quotes(normalize_text("\n".join(page.extract_text() for page in reader.pages)))
    for dish_name in dish_mapping:
        if normalize_quotes(dish_name) in full_text:
            all_found_full.add(dish_name)

recovered_quotes = all_found_full - all_found_ws
still_missing = set(dish_mapping.keys()) - all_found_full

print(f"After whitespace + quote normalization: {len(all_found_full)} / {len(dish_mapping)}")
print(f"Recovered {len(recovered_quotes)} more dishes (quote fix):")
for name in sorted(recovered_quotes):
    print(f"  + {name} (ID: {dish_mapping[name]})")

print(f"\nTruly missing (not in any PDF): {len(still_missing)}")
for name in sorted(still_missing):
    print(f"  - {name} (ID: {dish_mapping[name]})")

print(f"\n{'='*80}")
print("SUMMARY")
print(f"{'='*80}")
print(f"  Raw exact match:          {len(all_found)} / 287")
print(f"  + whitespace norm:        {len(all_found_ws)} / 287  (+{len(all_found_ws) - len(all_found)})")
print(f"  + curly quote norm:       {len(all_found_full)} / 287  (+{len(all_found_full) - len(all_found_ws)})")
print(f"\n\u2192 With trivial normalization, {len(all_found_full)/len(dish_mapping)*100:.1f}% of dishes are locatable by exact name match.")


After whitespace + quote normalization: 286 / 287
Recovered 1 more dishes (quote fix):
  + Sinfonia Quantica dell'Oceano Interstellare (ID: 238)

Truly missing (not in any PDF): 1
  - Mandragola e Radici (ID: 120)

SUMMARY
  Raw exact match:          280 / 287
  + whitespace norm:        285 / 287  (+5)
  + curly quote norm:       286 / 287  (+1)

→ With trivial normalization, 99.7% of dishes are locatable by exact name match.
→ Dish names from dish_mapping.json can serve as reliable anchors for parsing.
